In [1]:
import debugpy
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import os
import gzip
import numpy as np
import cell2location

ModuleNotFoundError: No module named 'scanpy'

In [3]:
# The model and output h5ad can be loaded later like this:
results_folder = './out/cell2loc/'

# create paths and names to results folders for reference regression and cell2location models
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'


adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref = sc.read_h5ad(adata_file)
inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']

def calcQCmat(andata):
    andata.var_names_make_unique()
    andata.var["mt"] = andata.var_names.str.startswith("mt-")
    andata.var["ribo"] = andata.var_names.str.startswith(("RPS", "RPL"))
    andata.var["hb"] = andata.var_names.str.contains("^HB[^(P)]")
    sc.pp.calculate_qc_metrics(andata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True)
    return andata
path_016 = "/data/kanferg/Sptial_Omics/playGround/Data/Visium_HD_Mouse_Brain_square_example/square_016um"
andata016_ = sc.read_visium(path=path_016)
andata016 = calcQCmat(andata016_)
andata016 = andata016[:,andata016.var.n_cells_by_counts > 50]
andata016.raw = andata016.copy()
sc.pp.normalize_total(andata016)
sc.pp.log1p(andata016)
log1p_data = andata016.X.todense()
sc.pp.highly_variable_genes(andata016)
sc.pp.scale(andata016)
andata016.obsm['spatial'] = np.array(andata016.obsm['spatial'], dtype=np.float64)
sc.pp.pca(andata016, n_comps=10)
sc.pp.neighbors(andata016)
sc.tl.leiden(andata016, key_added="clusters" ,resolution=0.7 , directed=False, n_iterations=2)


andata016.X = andata016.raw.X
andata016.var.set_index('gene_ids', drop=True, inplace=True)

intersect = np.intersect1d(andata016.var_names, inf_aver.index)
adata_vis = andata016[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

/usr/local/apps/cell2location/conda/envs/0.1.3/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/apps/cell2location/conda/envs/0.1.3/lib/python3.9/site-packages/scanpy/readwrite.py:413: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  positions = pd.read_csv(files['tissue_positions_file'], header=None)


In [4]:
inf_aver

,fibroblast,ependymal cell,endothelial cell,astrocyte,oligodendrocyte,microglial cell,erythrocyte,neuron,tanycyte,oligodendrocyte precursor cell,mural cell,pituitary gland cell,hypendymal cell
ENSMUSG00000000001,0.058990,0.279077,0.066953,0.037818,0.072204,0.036235,0.141449,0.051681,0.114949,0.060775,0.061798,0.255431,0.241630
ENSMUSG00000000056,0.019968,0.015756,0.016708,0.017777,0.064761,0.020044,0.069005,0.089046,0.017991,0.024033,0.021017,0.009871,0.162067
ENSMUSG00000000058,0.012424,0.218034,0.356125,0.010181,0.006120,0.004472,0.023368,0.025465,0.071475,0.265525,0.110415,0.023100,0.015309
ENSMUSG00000000078,0.152146,0.352939,0.723024,0.019676,0.063019,0.262876,0.082438,0.102375,0.296396,0.099048,0.115994,0.627748,0.799273
ENSMUSG00000000085,0.036610,0.030136,0.034895,0.044892,0.019203,0.010207,0.016939,0.100772,0.035266,0.036396,0.015850,0.087626,0.045606
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000117406,0.012331,0.002120,0.000286,0.002789,0.000054,0.000125,0.025317,0.025617,0.008223,0.005332,0.000606,0.005714,0.030791
ENSMUSG00000117679,0.000790,0.003974,0.000342,0.000106,0.002277,0.000107,0.081745,0.054330,0.000592,0.001228,0.001241,0.009385,0.024066
ENSMUSG00000117924,0.028883,0.043840,0.000435,0.022408,0.055767,0.000308,0.139751,0.079949,0.043310,0.077956,0.005776,0.043587,0.064498
ENSMUSG00000118332,0.001034,0.002343,0.002233,0.000767,0.003932,0.001233,0.044243,0.006884,0.001685,0.002156,0.005013,0.037763,0.022410


In [22]:
import pickle
path_out = './pymc_model'
df = pd.DataFrame(data=adata_vis.X.todense(), columns=adata_vis.var.index.tolist())
with open(os.path.join(path_out, 'brain_16mm_vshd.pkl'), 'wb') as buff:
            pickle.dump({'dsg':df,'wsf':inf_aver}, buff)